There is a surprising difference between the TMRCA estimates by Mendez et al. and my numbers, on the same exome capture data.

I suspect the difference is due to different filters. I'm, for instance, requiring >= 3X coverage for all data, including the exome capture (which is 3.3X). What about Mendez? Their paper does not mention explicitly any lower cutoff for coverage.

Mendez published his mpileup output. Let's compare this data to the exome genotype calls that I made myself.

In [1]:
library(tidyverse)
library(here)

devtools::load_all(".")

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1       ✔ purrr   0.3.2  
✔ tibble  2.1.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
here() starts at /Users/martin_petr/projects/ychr
Loading ychr


In [2]:
mendez <- read_tsv(here("input/mendez_counts.txt"),
                   col_names = c("chrom", "pos", "ref", "coverage", "pileup", "qual")) %>%
    mutate(ref = toupper(ref)) %>%
    select(-qual) %>%
    mutate(pileup = count_bases(pileup, ref)) %>% 
    unnest %>%
    select(-ref)

Parsed with column specification:
cols(
  chrom = col_character(),
  pos = col_double(),
  ref = col_character(),
  coverage = col_double(),
  pileup = col_character(),
  qual = col_character()
)


In [3]:
nrow(mendez)

[1] 146

In [4]:
mendez %>% head

chrom,pos,coverage,a,c,g,t
<chr>,<dbl>,<dbl>,<int>,<int>,<int>,<int>
Y,2710154,1,0,0,1,0
Y,2712091,6,6,0,0,0
Y,2722524,1,0,0,1,0
Y,2751547,1,0,0,0,1
Y,2844774,1,0,1,0,0
Y,2847322,9,0,0,0,9


In [4]:
mendez %>% head

chrom,pos,ref,coverage,qual,a,c,g,t
Y,2710154,G,1,<,0,0,1,0
Y,2712091,T,6,]]]MB@,6,0,0,0
Y,2722524,A,1,C,0,0,1,0
Y,2751547,C,1,],0,0,0,1
Y,2844774,T,1,<,0,1,0,0
Y,2847322,T,9,]]]]]]]]],0,0,0,9


In [6]:
neand <- read_vcf(here("data/vcf/exome_elsidron1.vcf.gz"), mindp = 3, maxdp = 0.975) %>%
    select(chrom, pos, REF, ALT, elsidron1)

ERROR: Error in .io_check_exists(path(con)): file(s) do not exist:
  '/Users/martin_petr/projects/ychr/data/vcf/exome_elsidron1.vcf.gz'


In [6]:
df <- full_join(neand, mendez)

Joining, by = c("chrom", "pos")


In [7]:
df %>% filter((a > 0 & a < 3) | (c > 0 & c < 3) | (g > 0 & g < 3) | (t > 0 & t < 3))

chrom,pos,REF,ALT,elsidron1,ref,coverage,qual,a,c,g,t
Y,14923532,T,C,1,T,2,]],0,2,0,0
Y,21646196,A,G,1,A,2,]M,0,0,2,0
Y,2710154,NA,NA,NA,G,1,<,0,0,1,0
Y,2722524,NA,NA,NA,A,1,C,0,0,1,0
Y,2751547,NA,NA,NA,C,1,],0,0,0,1
Y,2844774,NA,NA,NA,T,1,<,0,1,0,0
Y,4967724,NA,NA,NA,A,1,],0,1,0,0
Y,5092904,NA,NA,NA,T,1,\,0,1,0,0
Y,5206217,NA,NA,NA,T,2,D[,0,0,0,2
Y,5605569,NA,NA,NA,C,1,],0,0,1,0


In [12]:
df %>% filter(!is.na(ref)) %>% print(n = Inf)

# A tibble: 146 x 12
    chrom    pos REF   ALT   elsidron1 ref   coverage qual      a     c     g
    <chr>  <dbl> <chr> <chr>     <dbl> <chr>    <dbl> <chr> <int> <int> <int>
  1 Y     2.71e6 T     A             1 T            6 ]]]M…     6     0     0
  2 Y     2.85e6 T     ""            0 T            9 ]]]]…     0     0     0
  3 Y     6.74e6 T     ""            0 T           10 ]]]]…     0     0     0
  4 Y     6.93e6 G     C             1 G            5 ]]]]]     0     5     0
  5 Y     7.56e6 G     ""            0 G            4 ]]]]      0     0     4
  6 Y     9.15e6 A     G             1 A            6 ]]]]…     0     0     6
  7 Y     9.39e6 A     ""            0 A           10 ]]]]…    10     0     0
  8 Y     9.39e6 A     ""            0 A            7 ]]]]…     7     0     0
  9 Y     9.39e6 T     ""            0 T            7 ]]]]…     0     0     0
 10 Y     9.91e6 C     ""            0 C            6 ]]]]…     0     6     0
 11 Y     1.40e7 C     A             1 C   

In [37]:
neand <- read_gt("../data/vcf/merged2_exome.vcf.gz") %>% select(chrom, pos, REF, ALT, elsidron1, mendezelsidron1)

In [38]:
df <- full_join(neand, mendez)

Joining, by = c("chrom", "pos")


In [40]:
df %>% filter(ALT != "", is.na(coverage)) %>% arrange(ALT) %>% print(n = Inf)

# A tibble: 3,047 x 11
     chrom    pos REF   ALT   elsidron1 mendezelsidron1 coverage     a     c
     <chr>  <dbl> <chr> <chr>     <dbl>           <dbl>    <dbl> <int> <int>
   1 Y     2.66e6 T     A             0               0       NA    NA    NA
   2 Y     2.66e6 G     A             0               0       NA    NA    NA
   3 Y     2.66e6 G     A             0               0       NA    NA    NA
   4 Y     2.71e6 C     A             0               0       NA    NA    NA
   5 Y     2.71e6 G     A             0               0       NA    NA    NA
   6 Y     2.71e6 G     A             0               0       NA    NA    NA
   7 Y     2.71e6 T     A             0               0       NA    NA    NA
   8 Y     2.71e6 G     A             0               0       NA    NA    NA
   9 Y     2.72e6 G     A             0               0       NA    NA    NA
  10 Y     2.72e6 G     A             0               0       NA    NA    NA
  11 Y     2.73e6 G     A             0              

In [34]:
df %>% filter(!is.na(coverage)) %>% arrange(ALT) %>% print(n = Inf)

# A tibble: 146 x 10
    chrom      pos REF   ALT   elsidron1 coverage     a     c     g     t
    <chr>    <dbl> <chr> <chr>     <dbl>    <dbl> <int> <int> <int> <int>
  1 Y      2710154 G     ""            0        1     0     0     1     0
  2 Y      2847322 T     ""            0        9     0     0     0     9
  3 Y      6736327 T     ""            0       10     0     0     0    10
  4 Y      7556582 G     ""            0        4     0     0     4     0
  5 Y      7557358 C     ""            0        2     0     2     0     0
  6 Y      9005137 T     ""            0        1     0     0     0     1
  7 Y      9386241 A     ""            0       10    10     0     0     0
  8 Y      9386252 A     ""            0        7     7     0     0     0
  9 Y      9386255 T     ""            0        7     0     0     0     7
 10 Y      9745262 A     ""            0        2     2     0     0     0
 11 Y      9905318 C     ""            0        6     0     6     0     0
 12 Y     1409003